In [17]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD")
}

In [39]:
def get_db_connection():
    """Устанавливает соединение с базой данных"""
    return psycopg2.connect(**DB_CONFIG)

def get_table_columns(conn, table_name='news', schema='public'):
    """Получает список колонок в таблице"""
    with conn.cursor() as cur:
        cur.execute(f"""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = '{schema}' AND table_name = '{table_name}';
        """)
        return cur.fetchall()

def get_sample_data(conn, table_name='news', schema='public', limit=1000):
    """Получает образец данных из таблицы"""
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {schema}.{table_name} LIMIT {limit};")
        columns = [desc[0] for desc in cur.description]
        data = cur.fetchall()
        return pd.DataFrame(data, columns=columns)



In [40]:
try:
    conn = get_db_connection()
    print("Успешное подключение к базе данных")
    
    columns = get_table_columns(conn)
    print("\nСтруктура таблицы news:")
    for col_name, col_type in columns:
        print(f"{col_name}: {col_type}")
    
    print("\nПример данных:")
    df = get_sample_data(conn)
    display(df.tail())
    
except Exception as e:
    print(f"Ошибка: {e}")
finally:
    if 'conn' in locals():
        conn.close()
        print("\nСоединение с базой данных закрыто")

Успешное подключение к базе данных

Структура таблицы news:
id: integer
channel_id: integer
link_to_news: character varying
message: character varying
summary: character varying
params: json
published_at: timestamp without time zone

Пример данных:


,id,channel_id,link_to_news,message,summary,params,published_at
96,61,2,https://t.me/testtesttest139/120,Klkk,Спилки и ключи.,"[{'ru_content': 'economy', 'ru_title': None}]",2025-03-25 13:08:20.907566
97,60,2,https://t.me/testtesttest139/119,Ghhhhh,,"[{'ru_content': 'conflicts', 'ru_title': None}]",2025-03-25 13:08:18.640645
98,99,2,https://t.me/testtesttest139/153,**К Р А С И В О:** **новая столица Египта выгл...,В настоящее время в Каире строится новая столи...,"[{'ru_content': 'society', 'ru_title': 0}]",2025-03-25 14:11:01.189949
99,100,2,https://t.me/testtesttest139/154,Китайские автомобили Li и Zeekr не смогут появ...,Джеки и Джейки не смогут появиться в России ра...,"[{'ru_content': 0, 'ru_title': 'economy'}]",2025-03-25 14:13:29.564500
100,101,4,https://t.me/bbbreaking/203168,❗️**МВД РФ объявило в розыск политолога Екатер...,МВД объявило в розыск политолога Екатерину Шул...,"[{'ru_content': 0, 'ru_title': 'politics'}]",2025-03-25 14:31:25.786651



Соединение с базой данных закрыто
